In [1]:
import os
#from google.colab import drive
#drive.mount('/content/drive')
#os.chdir('/content/drive/MyDrive/Github/dus_mm/')
import numpy as np
import pandas as pd
import utils
import glob
from typing import List, Callable, Optional
from functools import reduce
os.chdir('data2')

columnsX = ['freq','AX','BX','BY','CY','CX','DX','DY','AY']
columnsY = ['AX','BX','BY','CY','CX','DX','DY','AY','AQ','AL','BQ','BL','CQ','CL','DQ','DL']
columnsYout = ['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']

In [2]:
X = pd.DataFrame()
y = pd.DataFrame()
numb_files = len(glob.glob('data_output*'))
for i in glob.glob('data_output*'): 
    numb = i[11]
    if (numb == 'data_output.txt'):
        numb = ''
    y = pd.concat([y,pd.read_table(os.path.join('data_output%s.txt' % numb), header = 0, names = columnsY, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float64')], ignore_index = 1)
    X = pd.concat([X,pd.read_table(os.path.join('data_input%s.txt' % numb), header = 0, names = columnsX, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float64')], ignore_index = 1) 
    
nelem = 64
freq_bias = [252.92, 97.13, 87.06, 57.57, 12.55, 3.40, -4.78, -11.92]
#freq_bias = [9.8,   4.9,   1.03, -12.05]
drop_ind = y.loc[(y > nelem).any(axis = 1)].index

X = utils.init_in(X, freq_bias)
# y = utils.decomposition(y)

# drop_ind.append(y.loc[(y == 0).all(axis = 1)].index)
# X = X.drop(drop_ind).reset_index(drop=True)
# y = y.drop(drop_ind).reset_index(drop=True)

In [3]:
X[]

,freq1,phaseA1,phaseB1,phaseC1,phaseD1,AXBX1,BYCY1,CXDX1,DYAY1,freq2,...,DYAY7,freq8,phaseA8,phaseB8,phaseC8,phaseD8,AXBX8,BYCY8,CXDX8,DYAY8
number,,,,,,,,,,,,,,,,,,,,,
0,0.088,0.997,0.000,0.998,0.000,0.952,1.000,0.872,0.834,0.986,...,0.030,0.504,0.681,0.416,0.789,0.515,0.141,0.006,0.011,1.000
1,0.000,1.000,0.000,0.999,0.000,0.913,0.884,1.000,0.975,0.748,...,1.000,0.069,0.879,0.186,0.421,0.888,0.304,1.000,0.056,0.111
2,0.480,1.000,0.001,1.000,0.001,0.994,0.899,0.907,1.000,0.739,...,0.486,0.744,0.371,0.330,0.973,0.771,0.055,0.633,1.000,0.231
3,0.617,1.000,0.000,1.000,0.000,0.984,0.954,0.955,1.000,0.097,...,0.513,0.150,0.964,0.827,0.996,0.724,0.324,1.000,0.831,0.187
4,0.700,1.000,0.001,1.000,0.001,0.925,1.000,0.980,0.914,1.000,...,0.283,0.190,0.767,0.186,0.978,0.363,0.262,0.085,0.119,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146,0.595,1.000,0.000,0.999,0.002,0.871,0.901,1.000,0.904,0.690,...,0.619,0.481,0.083,0.082,0.996,0.691,0.491,0.201,0.160,1.000
2147,0.525,1.000,0.000,1.000,0.000,0.969,0.946,1.000,0.974,0.698,...,1.000,0.000,0.213,0.018,0.688,0.952,0.782,1.000,0.142,0.212
2148,1.000,1.000,0.001,1.000,0.000,0.987,0.932,0.974,1.000,0.989,...,0.366,0.654,0.857,0.227,0.268,0.680,0.229,1.000,0.196,0.052


In [ ]:
alfa = 1
koef = np.zeros(len(freq_bias))
X1 = utils.init_in(X.copy(), freq_bias)
for i in range (100):
    for n in range(len(freq_bias)):
        koef[n] = (0.5 - X1['freq' + str(n+1)].mean())
    freq_bias = freq_bias + koef * alfa
    X1 = utils.init_in(X.copy(), freq_bias)

In [ ]:
def compute_loss(prediction, y, diff: bool = False):
#     if (loss_funtion == 'mean_squared_error'):
    if (diff == True):
        loss = prediction - y
    else:
        loss = np.sum(np.sum(np.power((prediction - y), 2), axis = None) )/ (2 * y.shape[0])
#     elif (loss_funtion == 'loglos'):
#         if (diff == True):
#             loss = (prediction - y)/((1 - prediction) * prediction)
#         else:
#             loss = - np.sum(y * np.log(prediction) + (1 - y) * np.log(1 - prediction))/y.shape[0]     
    return loss

def activation(name: str, x: np.ndarray, diff: bool = False):
        if (name == 'relu'):
            if (diff == True):
                x[x >= 0] = 1
                x[x < 0] = 0
            else:
                x[x < 0] = 0
        elif (name == 'linear'):
            if (diff == True):
                x = 1
            else:
                x = x
        elif (name == 'sigmoid'):
            if (diff == True):
                x = activation(name, x) * (1 - activation(name, x))
            else:
                x = 1/(1 + np.exp(-x))
        return x

In [ ]:
class Layer:
    def __init__(self, previous_layer,  neurons: int, activation_name: str = 'linear') -> None:
        self.previous_layer = previous_layer
        self.next_layer = None
        self.neurons: int = neurons
        self.activation_name = activation_name
        self.__weights: np.ndarray 
        self.weights_cache = self.initialization_weights()
        self.outputs_cache: np.ndarray
        self.deltas_cache: np.ndarray 
        self.gradients_cache: np.ndarray
    
    def initialization_weights(self) -> np.ndarray:
        if self.previous_layer is None:
            self.weights_cache = np.array([]) 
        else:
            gamma = np.sqrt(6) / np.sqrt(self.previous_layer.neurons + self.neurons)
            self.weights_cache = np.random.rand(self.neurons,self.previous_layer.neurons + 1) * 2 * gamma - gamma          
            return self.weights_cache
    
    def calculate_outputs(self, inputs: np.ndarray) -> np.ndarray:
        if self.previous_layer is None:
            self.outputs_cache = inputs
        else:
            inputs = np.concatenate((np.ones([inputs.shape[0],1]), inputs), axis = 1)
            self.outputs_cache =  activation(self.activation_name, inputs @ self.weights_cache.T)       
        return self.outputs_cache
    
    def calculate_deltas_and_gradients(self, y: np.ndarray) -> None:
        inputs = self.previous_layer.outputs_cache
        inputs = np.concatenate((np.ones([inputs.shape[0],1]), inputs), axis = 1)
        if self.next_layer is None:
            self.deltas_cache = compute_loss(self.outputs_cache, y, diff = True) * activation(self.activation_name, np.dot(inputs, self.weights_cache.T) , diff = True)
        else:
            self.deltas_cache = np.dot(self.next_layer.deltas_cache, self.next_layer.weights_cache[:,1:]) * activation(self.activation_name, np.dot(inputs, self.weights_cache.T), diff = True)
        self.gradients_cache = (1 / y.shape[0]) * np.dot(self.deltas_cache.T, inputs)
    
    def _record_weights(self):
        self.__weights = self.weights_cache
    
    def _playback_weights(self):
        self.weights_cache = self.__weights 

In [ ]:
class NN_model: 
    def __init__(self, input_dimension: int, loss = 'mean_squared_error'):
        self.sum_neurons = 0
        self.layers: List[Layer] = [] 
        self.add_layer(input_dimension)
        self.loss = loss
        self.optimizer = 'adam' #'adam'
        self.max_epoch = 1000
        self.chunk_size = 32
        self.beta_1 = 0.9
        self.beta_2 = 0.99
        self.epsilon = 1e-8
        self.learning_rate = 0.01
        self.regularization = None #'l2'
        self.best_loss_val = 1e10
    
    def add_layer(self, neurons: int, activation) -> None:
        if not self.layers:
            new_layer = Layer(None, neurons)
        else:
            new_layer = Layer(self.layers[-1], neurons, activation)
            self.layers[-1].next_layer = new_layer   
        self.layers.append(new_layer)
        self.sum_neurons += neurons 
    
    def calculate_outputs(self, inputs:  np.ndarray) -> np.ndarray:
         return reduce(lambda inputs, layer: layer.calculate_outputs(inputs), self.layers, inputs)  
    
    def backpropagate(self, y):
        for i in range(len(self.layers) - 1, 0, -1):
            self.layers[i].calculate_deltas_and_gradients(y)          

    def gradient_checking(self, x, y) -> None:
        e: float = 1e-4
        numgrad = np.array([])
        grad = np.array([])
        self.calculate_outputs(x)
        self.backpropagate(y)
        j = 0
        for i in range(1, len(self.layers)):
            grad = np.concatenate([grad, self.layers[i].gradients_cache.ravel()])
            numgrad = np.concatenate([numgrad, np.zeros(self.layers[i].weights.size)])
            perturb = np.diag(e * np.ones(self.layers[i].weights.size))
            weights = self.layers[i].weights.copy()
            for k in range(self.layers[i].weights.size):
                self.layers[i].weights += perturb[:,k].reshape(self.layers[i].weights.shape)
                predicted_plus  = self.calculate_outputs(x)
                self.layers[i].weights -= perturb[:,k].reshape(self.layers[i].weights.shape)
                predicted_minus = self.calculate_outputs(x)
                numgrad[j] = (compute_loss(predicted_plus, y) - compute_loss(predicted_minus, y))/(2 * e)
                j += 1
            self.layers[i]._playback_weights(self):
        diff = np.linalg.norm(numgrad - grad)/np.linalg.norm(numgrad + grad)
        print('Relative Difference: %g' % diff) 
     
#     def optimize(self, X_train, y_train, X_val, y_val, lambda_new = 0 , theta_new = None):
#         if (theta_new is None):
#             theta_new = self.theta
#         for epoch in range(self.max_epoch):
#             m = np.zeros(self.theta.shape)
#             v = np.zeros(self.theta.shape)
#             lst =  np.arange(y_train.shape[0])
#             np.random.shuffle(lst)
#             chunks = np.array([lst[i: i + self.chunk_size] for i in range(0, y_train.shape[0], self.chunk_size)])
#             for t, chunk in enumerate(chunks):
#                 xi = X_train[chunk]
#                 yi = y_train[chunk]
#                 self.calculate_outputs(xi)
#                 self.backpropagate(yi)
#                 if (self.optimizer == 'adam'):
#                     for i in range(1, len(self.layers)):
#                         m = self.beta_1 * m + (1 - self.beta_1) * layer[i].calculate_gradients()
#                         v = self.beta_2 * v + (1 - self.beta_2) * np.power(grad, 2)
#                         m_hat = m / (1 - np.power(self.beta_1, t + 1))
#                         v_hat = v / (1 - np.power(self.beta_2, t + 1))
#                         layers[i].weights = layers[i].weights - self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon) 
#                 elif (self.optimizer == 'gradient_descent'):
#                     theta_new = theta_new - self.learning_rate * grad
#             a_out_train = self.feedforward(X_train, theta_new)
  
#             loss_train = self.compute_loss(a_out_train, y_train, theta_new)# + lambda_new * np.sum(np.power(self.nulling(theta_new), 2))/2
#             a_out_val = self.feedforward(X_val, theta_new)
#             loss_val = self.compute_loss(a_out_val, y_val, theta_new) #+ lambda_new * np.sum(np.power(self.nulling(theta_new), 2))/2
            
#             if (self.best_loss_val > loss_val):  
#                 self.theta = theta_new
#                 self.best_loss_val = loss_val        
                
#             print(f'{np.round(loss_train,2)}    {np.round(loss_val,2)}    {epoch}')
#         return loss_train, loss_val
    
#     def __repr__(self):
#         return str(len(self.sequential))

In [ ]:
dus_mm = NN_model(X.shape[1], 'loglos')
dus_mm.add_layer(16, 'sigmoid')
dus_mm.add_layer(16, 'sigmoid')
dus_mm.add_layer(y.shape[1], 'sigmoid')

In [ ]:
dus_mm.calculate_outputs(X)

In [ ]:
dus_mm.backpropagate(y)

In [ ]:
dus_mm.gradient_checking(X, y)

In [ ]:
print(dus_mm.layers[2].output_cache)

In [ ]:
import scipy.io
mat = scipy.io.loadmat('mnist.mat')
X = mat['X']
y = mat['y']
y[y == 10] = 0
y0 = np.zeros((y.size, 10))
for i in range(y.shape[0]):
    y0[i,y[i]] = 1
y = y0

In [ ]:
dus_mm = NN_model(X.shape[1])
dus_mm.add(Layer(128, 'relu'))
dus_mm.add(Layer(128, 'relu'))
dus_mm.add(Layer(y.shape[1], 'linear'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# накидываем тысячу точек от -3 до 3
X = np.linspace(-3, 3, 1000).reshape(-1, 1)

# задаём линейную функцию, которую попробуем приблизить нашей нейронной сетью
def f(x):    
    return 2 * np.sin(x) + 5

f = np.vectorize(f)

# вычисляем вектор значений функции
y = f(X)

# отрисовываем результат приближения нейросетью поверх исходной функции
plt.scatter(X_train, y, color='black', antialiased=True)
#plt.plot(, model.predict(x), color='magenta', linewidth=2, antialiased=True)
plt.show()

# выводим веса на экран
# for layer in model.layers:
#     weights = layer.get_weights()
#     print(weights)

In [ ]:
dus_mm = NN_model(X.shape[1], 'mean_squared_error')
dus_mm.add(Layer(y.shape[1], 'linear'))

In [ ]:
X = np.array(X)
y = np.array(y)
lambda_vec = [0]

nfold = 5
slice_fold = int(y.shape[0] // nfold)
lst =  np.arange(y.shape[:(5 * slice_fold)][0])
np.random.shuffle(lst)
list_fold = np.array([lst[i * slice_fold:(i + 1) * slice_fold] for i in range(nfold)])

train_error = np.zeros(nfold)
val_error = np.zeros(nfold)
train_reg = np.zeros(len(lambda_vec))
val_reg = np.zeros(len(lambda_vec))
theta_new = dus_mm.theta.copy()

for k in range(len(lambda_vec)):
    theta_new
    for i in range(nfold):
        X_train = X[np.delete(list_fold, i, 0).ravel()]
        y_train = y[np.delete(list_fold, i, 0).ravel()]
        X_val = X[list_fold[i]]
        y_val = y[list_fold[i]]
        train_error[i], val_error[i] = dus_mm.optimize(X_train, y_train, X_val, y_val, lambda_vec[k], theta_new)
    train_reg[k] = np.mean(train_error)  
    val_reg[k] = np.mean(val_error)

['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']
  -1., -0., -2.,  1., 34., 65.,  5., 23., 81., -2., 18., 21
   2.,  1.,  1.,  0., 15., 65., 24.,  2., 83.,  1.,  7.,  4.
   0.,  1., -1.,  10.,21., 63., 26.,  6., 86.,  -1., 12.,-20.
      
   1.,  1.,  3.,  7., 47., 15., 20.,  48., 54.,  3.,-11., 60.

In [ ]:
XX = pd.read_table(os.path.join('data7A-1_2.txt'), header = 0, names = columnsX, index_col = False, sep='\s+', engine='python', dtype ="float")
XX = utils.init_in(XX,freq_bias)
yy = np.round(dus_mm.feedforward(np.array(XX))[:12])
#yy = np.abs(pd.DataFrame(yy.astype('float'), columns = columnsYout))
#yy = utils.integration(yy,nelem)
# predictions = NN_model.predict(XX)
# yy = np.abs(pd.DataFrame(predictions.astype('float'), columns = columnsYout))
#yy = utils.integration(yy,0.72*1.33,1.33) # перемычки 30 мкм вместо 40 мкм 1.3

In [ ]:
yy

In [ ]:
len(columnsYout)

In [ ]:
pd.DataFrame(yy.astype('float'), col)

In [ ]:
yy = utils.integration(np.round(dus_mm.feedforward(np.array(XX))[:12]),nmode)

In [ ]:
a_out_train = self.feedforward(X_train)

In [ ]:
dus_mm.form_output(X_val)

In [ ]:
X_val.shape

In [ ]:
# отрисовываем результат приближения нейросетью поверх исходной функции
plt.scatter(X_val, y_val, color='black', antialiased=True)
plt.plot(X_val, dus_mm.form_output(X_val,y_val), color='magenta', linewidth=2, antialiased=True)
plt.show()

In [ ]:
f = dus_mm.gradient_checking(X,y)

In [ ]:
lambda_vec = [0]
error_train = np.zeros(len(lambda_vec))
error_val = np.zeros(len(lambda_vec))
init_theta = dus_mm.theta

for i in range(len(lambda_vec)):
  dus_mm.theta = init_theta.copy()
  dus_mm.optimize(X_train, y_train, lambda_vec[i]) 
  a_out = dus_mm.feedforward(X_train)
  a_end = a_out[:(dus_mm.sequential[-1].neurons * y_train.shape[0])]
  error_train[i] = dus_mm.compute_loss(a_end, y_train)
  
  a_out = dus_mm.feedforward(X_val)
  a_end = a_out[:(dus_mm.sequential[-1].neurons * y_val.shape[0])]
  error_val[i] = dus_mm.compute_loss(a_end, y_val)

In [ ]:
plt.plot(lambda_vec, error_train, '-o', lambda_vec, error_val, '-o', lw=2)
plt.legend(['Train', 'Cross Validation'])
plt.xlabel('lambda')
plt.ylabel('Error')

print('lambda\t\tTrain Error\tValidation Error')
for i in range(len(lambda_vec)):
    print(' %f\t%f\t%f' % (lambda_vec[i], error_train[i], error_val[i]))

In [ ]:
a_out = dus_mm.feedforward(X_val)
a_end = a_out[:(dus_mm.sequential[-1].neurons * y_val.shape[0])]

In [ ]:
a_end

In [ ]:
a_out = dus_mm.feedforward(X_val)
a_end = a_out[:(dus_mm.sequential[-1].neurons * y_val.shape[0])]
loss = dus_mm.compute_loss(a_end, y_val.ravel())

In [ ]:
loss

In [ ]:
y[1000:1001]

In [ ]:
X[1000:1001]

In [ ]:
np.round(dus_mm.feedforward(np.array(X[1000:1001]))[:12])

In [ ]:
import scipy.io
mat = scipy.io.loadmat('mnist.mat')
X = mat['X']
y = mat['y']
y[y == 10] = 0
y0 = np.zeros((y.size, 10))
for i in range(y.shape[0]):
    y0[i,y[i]] = 1

In [ ]:
dus_mm = NN_model(X.shape[1], 'loglos')
dus_mm.add(Layer(128, 'sigmoid'))
dus_mm.add(Layer(128, 'sigmoid'))
dus_mm.add(Layer(y0.shape[1], 'sigmoid'))

In [ ]:
X = np.array(X)
y = np.array(y)
lst =  np.arange(y.shape[0])
np.random.shuffle(lst)
split_data = int(0.8 * y.shape[0])

X_train = X[lst[:split_data]]
y_train = y[lst[:split_data]]
X_val = X[lst[split_data:y.shape[0]]]
y_val = y[lst[split_data:y.shape[0]]]

In [ ]:
dus_mm.optimize(X_train, y_train, X_val, y_val )

In [ ]:
dus_mm.feedforward(X[600:601,:])[:10]

In [ ]:
y0[600]

In [ ]:
 a = [1,2,3]

In [ ]:
X